# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** *The type of supervised learning problem that this is, is classification. This is because we are looking for a discreet not continuous output, namely whether a student needs intervention or not.*

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [90]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [91]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features =  len(student_data.columns[:-1])

# TODO: Calculate passing students
n_passed = np.sum(student_data['passed'].map(lambda x: 1 if x=='yes' else 0))

# TODO: Calculate failing students
n_failed = np.sum(student_data['passed'].map(lambda x: 0 if x=='yes' else 1))

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/float(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [92]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [93]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [94]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
# 1. Decision Trees
### Real-world Applications
    * Used for building personal learning assistants and for classifying sleep signals
    * Used for remote sensing for pattern recognition
    * Used for the detection of physical particles
    * Used for analyzing amino acid sequences
    *  automatic induction of decision trees can be found in diagnosis, cardiology, psychiatry, gastroenterology, for detecting microcalcifications in mammography, to analyze Sudden Infant Death (SID) syndrome and for diagnosing thyroid disorders.
    * Automatic induction of decision trees was recently used for control of nonlinear dynamical systems
    * Helped in star-galaxy classification [372], determining galaxy counts [371] and discovering quasars [177] in the Second Palomar Sky Survey
    * Used for filtering noise from Hubble Space Telescope images
    * Used to non-destructively test welding quality, for semiconductor manufacturing, for increasing productivity, for material procurement method selection, to accelerate rotogravure printing, for process optimization in electrochemical machining, to schedule printed circuit board assembly lines, to uncover flaws in a Boeing manufacturing process and for quality control. 
### Advantages
    * Simple to understand and to interpret. Trees can be visualised.
    * Requires little data preparation. 
    * The cost of using the tree is O(log(n)).
    * Able to handle both numerical and categorical data. 
    * Able to handle multi-output problems.
    * Uses a white box model.
    * Possible to validate a model using statistical tests.
    * Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.
    
### Disadvatages
    * Decision-tree learners can create over-complex trees that do not generalise the data well. 
    *Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. 
    * The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts.
    * There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
    * Decision tree learners create biased trees if some classes dominate. 
### Why this is useful for us?
    * Some of this data might affect the data more so than not. It would be possible to get huge changes in Entropy, with a shallow tree. For Instance, we can expect Absenes to give us alot more information than age or sex. Also our data contains both numerical and categorical features which Decision Trees are good at handling. 
# 2. Support Vector Machines (SVM)
### Real-world Applications
    * Used to classify proteins with up to 90% of the compounds classified correctly. 
    * Used to recognize Hand-written characters.
    * Used for Classification of images.
    * Used in text and hypertext categorization as their application can significantly reduce the need for labeled training instances.
### Advantages
    * Effective in high dimensional spaces.
    * Still effective in cases where number of dimensions is greater than the number of samples.
    * Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
    * Versatile: different Kernel functions can be specified for the decision function. 
### Disadvatages
    * If the number of features is much greater than the number of samples, the method is likely to give poor performances.
    * SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation.
### Why this is useful for us?
    * Support Vector machines offers high versitility and handles high number of features. Our data high a high number of features which could be hard to fit with a 47-dimensional plane. The Advantage of the Kernel provides us is that we can linearly transformation that can make our data linearly seperable.
# 3. K-Nearest Neighbors
### Real-world Applications
    * Used in protein- protein interaction prediction.
    * Used in predicting gene expression.
    * Used in Computer Vision based Content Retrieval.(ie. Video)
### Advantages
    * Robust to noisy training data.
    * Effective if the training data is large.
### Disadvatages
    * Need to determine value of parameter K (number of nearest neighbors).
    * Distance based learning is not clear which type of distance to use and which attribute to use to produce the best results. 
    * Computation cost is quite high because we need to compute distance of each query instance to all training samples. Some indexing (e.g. K-D tree) may reduce this computational cost.
### Why this is useful for us?
    * K-Nearest neighbor is my prime choice. Even though we dont have a lot of data I believe that we will find regions of similarly classified data in this 48-dimensional space.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [95]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [96]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier()
clf_B = SVC()
clf_C = KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[0:100] 
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]


#train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
#train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0041 seconds.
F1 score for test set: 0.7068.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0017 seconds
Made predictions in 0.0078 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0051 seconds.
F1 score for test set: 0.7121.
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0037 seconds
Made predictions in 0.0112 seconds.
F1 score for training set: 0.8722.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.7482.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Trees**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0058 seconds |         0.0055 seconds |           1.0000 |          0.7154 |
| 200               |          0.0036 seconds |         0.0006 seconds |           1.0000 |          0.7188 |
| 300               |          0.0050 seconds |         0.0005 seconds |           1.0000 |          0.7273 |

** Classifer 2 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0037 seconds |         0.0043 seconds |           0.8591 |          0.7838 |
| 200               |          0.0065 seconds |         0.0031 seconds |           0.8693 |          0.7755 |
| 300               |          0.0129 seconds |         0.0027 seconds |           0.8692 |          0.7586 |

** Classifer 3 - K-Nearest Neighbors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0016 seconds |         0.0073 seconds |           0.7972 |          0.7068 |
| 200               |          0.0019 seconds |         0.0051 seconds |           0.8571 |          0.7121 |
| 300               |          0.0077 seconds |         0.0084 seconds |           0.8722 |          0.7482 |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Looking at the total run time we see that SVM can be quite expensive. However KNN and DT seem to be faster by a factor of 10. However out of KNN and DT, KNN seem to have a higher F1 Score even with untuned parameters. This method will give us relativly high accuracy and with fast speed. This algorithm will continue to be robust as the amount of data grows, even if it is very noisy. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** Using the SVM Classifier will be to expensive. It will either take a long time to teach our computers  about the students or very expensive hardware to do it fast. However if we were to use the KNN and DT Classifiers we could tacke the problem cheaply. Out of those 2 options KNN seem to be right more often even before we try to "help" the machines out. Even If we dont have all the factor that contribute to student failiure, our result will capture effectively relationships that do exist in our data. Even as we start to add more and more students to our databases our model should still hold. So imagine you gave me an address and ask me information about the occupants, but didnt allow me to talk to the occupants only their neighbors. Also I can not ask the neighbors directly about the occupant because either they dont know the information or wont tell me. Well If 30/40 of their neighbors are millenial parents, 35/40 of their neighbor are african american and 38/40 of the neighbor are Democrats, then I could guess with some certainty that the occupant of that address are young black parents who affliliate with the Democratic Party.  In the same way that real life neighborhoods share common data. Our algorithm make a neighborhood around the things we want to know. This is because what the KNN is doing under the hood is grabing a certain number of datapoint, K amount, that are the most similar in some way(ie. in the closet neighbors in their neighborhood). It looking at the information these neighboring data gives us and makes a guess about the datapoint we want to know about(the occupant). This is useful to us because we can enter information about a student. That information get an address in the data world. Then we will be able to see if his neighbors in this data world needed intervention or not. If his data-neighbors did then so does this student, and likewise in the converse case.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [97]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import DistanceMetric

# TODO: Create the parameters list you wish to tune
parameters = {'metric':('euclidean', 'manhattan', 'chebyshev', 'minkowski'),'n_neighbors':[1, 5*num_train/9], 'leaf_size':[1, 5*num_train/9],'p':[1,3], 'weights':('uniform','distance'), 'algorithm':('auto', 'ball_tree', 'kd_tree', 'brute')}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0085 seconds.
Tuned model has a training F1 score of 0.8142.
Made predictions in 0.0031 seconds.
Tuned model has a testing F1 score of 0.7662.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final model's F<sub>1</sub> score for training and testing is __ and __ respectively. This score is 0.018 higher than the untuned model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.